In [66]:
import duckdb
import pandas as pd

In [67]:
con = duckdb.connect(database='dot.duckdb', read_only=False)


In [68]:
con.execute("CREATE OR REPLACE TABLE users AS SELECT * FROM read_csv_auto('../data/users.csv');")
con.execute("CREATE OR REPLACE TABLE posts AS SELECT * FROM read_csv_auto('../data/posts.csv');")
con.execute("CREATE OR REPLACE TABLE impressions AS SELECT * FROM read_csv_auto('../data/feed_impressions.csv');")

# quick check
con.execute("SELECT COUNT(*) FROM users").fetchall()
con.execute("SELECT COUNT(*) FROM posts").fetchall()
con.execute("SELECT COUNT(*) FROM impressions").fetchall()


[(2007080,)]

# Harm Metrics Analysis

In this section, we quantify the impact of stolen posts using SQL:

1. % of posts that are stolen  
2. % of impressions captured by stolen posts  
3. Impression loss for original creators  
4. Harm segmentation by creator type and geography  


In [69]:
con.execute("""
    SELECT 
        COUNT(*) FILTER (WHERE is_stolen = TRUE) * 1.0 / COUNT(*) AS pct_stolen_posts
    FROM posts;
""").fetchall()


[(0.3379675604104601,)]

In [70]:
con.execute("""
    SELECT 
        SUM(CASE WHEN p.is_stolen THEN 1 ELSE 0 END) * 1.0 / COUNT(*) AS pct_stolen_impressions
    FROM impressions i
    JOIN posts p ON i.post_id = p.post_id;
""").fetchall()


[(0.33573499810670226,)]

In [71]:
import duckdb
import pandas as pd

con = duckdb.connect("dot.duckdb", read_only=False)


In [72]:
con.execute("CREATE OR REPLACE TABLE users AS SELECT * FROM read_csv_auto('../data/users.csv');")
con.execute("CREATE OR REPLACE TABLE posts AS SELECT * FROM read_csv_auto('../data/posts.csv');")
con.execute("CREATE OR REPLACE TABLE impressions AS SELECT * FROM read_csv_auto('../data/feed_impressions.csv');")


### Interpretation of Metrics

**1. Percent of Posts That Are Stolen:**  
Approximately **33–34%** of all posts in the dataset are stolen copies.  
This indicates that nearly *one-third* of content on DOT is duplicated rather than original.

**2. Percent of Impressions Captured by Stolen Posts:**  
Stolen posts receive around **33–34%** of all feed impressions.  
This suggests that duplicate content achieves a similar (or even higher) level of visibility compared to original posts.

Both metrics highlight a significant platform-level issue:  
stolen content competes directly with original creators for visibility, potentially reducing fairness and harming creator retention.


# Harm Ranking Across Content Groups

In [73]:
con.execute("""
WITH group_stats AS (
    SELECT
        p.group_id,
        SUM(1) AS total_impressions,
        SUM(CASE WHEN p.is_original THEN 1 ELSE 0 END) AS original_impressions
    FROM impressions i
    JOIN posts p ON i.post_id = p.post_id
    GROUP BY p.group_id
)
SELECT
    group_id,
    total_impressions,
    original_impressions,
    total_impressions - original_impressions AS lost_impressions
FROM group_stats
ORDER BY lost_impressions DESC
LIMIT 20;
""").fetchdf()


,group_id,total_impressions,original_impressions,lost_impressions
0,1131,4532.0,869.0,3663.0
1,1494,4686.0,1162.0,3524.0
2,637,4386.0,868.0,3518.0
3,444,3915.0,715.0,3200.0
4,1863,3856.0,753.0,3103.0
5,790,3873.0,808.0,3065.0
6,892,3768.0,765.0,3003.0
7,485,3912.0,944.0,2968.0
8,1967,4220.0,1255.0,2965.0
9,1603,4139.0,1182.0,2957.0


This query identifies groups where original posts lose the highest number of impressions to their stolen copies.  
The `lost_impressions` metric gives us a direct view into how much visibility original creators lose due to duplication.

This helps answer:
- Which content types or creators are most harmed?
- How large is the exposure gap between original and stolen versions?


# Harm by Creator Type and Geography

In [74]:
con.execute("""
WITH group_harm AS (
    SELECT
        p.group_id,
        MIN(CASE WHEN p.is_original THEN p.author_id END) AS original_author_id,
        SUM(1) AS total_impressions,
        SUM(CASE WHEN p.is_original THEN 1 ELSE 0 END) AS original_impressions
    FROM impressions i
    JOIN posts p ON i.post_id = p.post_id
    GROUP BY p.group_id
)
SELECT
    u.creator_type,
    u.country,
    SUM(total_impressions - original_impressions) AS lost_impressions
FROM group_harm g
JOIN users u ON g.original_author_id = u.user_id
GROUP BY u.creator_type, u.country
ORDER BY lost_impressions DESC;
""").fetchdf()


,creator_type,country,lost_impressions
0,casual,US,179620.0
1,casual,IN,102240.0
2,casual,BR,87462.0
3,casual,GB,77334.0
4,casual,CA,43205.0
5,influencer,US,42009.0
6,influencer,IN,36796.0
7,business,IN,21221.0
8,business,US,17773.0
9,influencer,GB,13954.0


This segmentation helps us understand how duplication disproportionately affects different groups.

Typical insights we expect:
- **Casual creators** often lose the most impressions because their posts are easier to copy.
- Certain **countries** may experience more stolen content due to higher platform activity or creator saturation.
- **Influencers**, despite having more engagement, may lose a large share of impressions to copycats.

This kind of analysis directly informs ranking policy and creator support initiatives.


In [75]:
harm_by_country = con.execute("""
WITH group_harm AS (
    SELECT
        p.group_id,
        MIN(CASE WHEN p.is_original THEN p.author_id END) AS original_author_id,
        SUM(1) AS total_impressions,
        SUM(CASE WHEN p.is_original THEN 1 ELSE 0 END) AS original_impressions
    FROM impressions i
    JOIN posts p
        ON i.post_id = p.post_id
    GROUP BY p.group_id
)
SELECT
    u.country,
    SUM(total_impressions - original_impressions) * 1.0 / SUM(total_impressions) AS avg_impression_loss_pct
FROM group_harm g
JOIN users u
    ON g.original_author_id = u.user_id
GROUP BY u.country
ORDER BY avg_impression_loss_pct DESC;
""").fetchdf()

harm_by_country.to_csv("../data/harm_by_country_summary.csv", index=False)
harm_by_country.head()


,country,avg_impression_loss_pct
0,GB,0.345959
1,BR,0.343364
2,CA,0.342307
3,US,0.334334
4,IN,0.324301


In [76]:
harm_by_country.to_csv("../data/harm_by_country_summary.csv", index=False)
